## Imports & Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import re
import ast
from datasets import load_dataset
from tqdm import tqdm
import helpers
import soft_prompting
import importlib
importlib.reload(helpers)
importlib.reload(soft_prompting)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# CONSTANTS
NUM_EPOCHS_FT = 100
NUM_EPOCHS_KD = 100
BATCH_SIZE = 32

## Get Data and preprocess it

In [ ]:
### train data
# data with label and image data
df_train_label = pd.DataFrame(load_dataset('derek-thomas/ScienceQA', split='train'))
df_train_label = df_train_label[df_train_label['solution'] != ''].reset_index()
df_train_image = df_train_label['image']
df_train_label['input'] = df_train_label.apply(lambda row: helpers.build_prompt(row)[0], axis=1)
df_train_label[['input', 'answer', 'explanation']] = df_train_label[['input', 'answer', 'solution']]

# data from Gemini for KD
df_train_gemini = pd.read_csv('gemini_1_5_flash_output_train.csv', sep="\t")[['input', 'answer', 'explanation']]

In [ ]:
### val data
df_val = pd.DataFrame(load_dataset('derek-thomas/ScienceQA', split='validation'))

In [ ]:
### Build DataLoader
# ToDo

## Functions for model training

In [ ]:
def train(dataloader_train, dataloader_val):
    train_errors = []
    val_errors = []
    for epoch in tqdm(range(NUM_EPOCHS_FT)):
        error = 0
        num_samples = 0
        # for X, y in dataloader_train:
            # forward
            # loss = ...
            # loss.backward()
            # optimizer.step()
            # error += loss.item() * X.size(0)
            # num_samples += X.size(0)
        error /= num_samples
        print(f'Error after epoch {epoch}: {error}')
        train_errors.append((epoch, error))
        if epoch % 10:
            val_error = 0
            num_samples
            # for X, y in dataloader_val:
                # forward
                # loss = ...
                # loss.backward()
                # optimizer.step()
                # val_error += loss.item() * X.size(0)
                # num_samples += X.size(0)
            val_error /= num_samples
            print(f'Validation error after epoch {epoch}: {val_error}')
            val_errors.append((epoch, val_error))
    return train_errors_ft, val_error

In [ ]:
def visualize_error(train_errors, val_errors):
    plt.plot(zip(*train_errors), label="Train Error", marker="o", linestyle="-")
    plt.plot(zip(*val_errors), label="Train Error", marker="o", linestyle="-")
    plt.title("Train and Validation Error over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Error")
    plt.show()

## Soft prompting using labels

In [ ]:
model = None # ToDo
model_fine_tuned = soft_prompting.MultimodalSoftPrompting.from_pretrained(model)

In [ ]:
train_errors_ft, val_errors_ft = train(dataloader_train_ft, dataloader_val_ft)

In [ ]:
visualize_error(train_errors_ft, val_errors_ft)

## Knowledge Distillation

In [ ]:
model = None # ToDo
model_knowledge_distillation = soft_prompting.MultimodalSoftPrompting.from_pretrained(model)

In [ ]:
train_errors_ft, val_errors_ft = train(dataloader_train_kd, dataloader_val_kd)

In [ ]:
visualize_error(train_errors_kd, val_errors_kd)